<a href="https://colab.research.google.com/github/Engineer-Ayesha-Shafique/Brain-Tumor-Segmentation-and-Detection-using-UNET-and-Watershed-in-Python/blob/main/Brain_Tumor_Segmentation_and_Detection_using_UNET_and_Watershed_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive' ,force_remount=True)

Mounted at /content/drive


#UNET

https://www.kaggle.com/datasets/nikhilroxtomar/brain-tumor-segmentation


In [2]:
!pip install tensorflow

In [8]:
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, Activation,
    MaxPooling2D, Conv2DTranspose, concatenate
)
from tensorflow.keras.models import Model

def conv_block(x, filters, kernel_size=(3,3), padding="same"):
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(filters, kernel_size, padding=padding)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def build_unetplusplus(input_shape=(224,224,3), num_classes=5):
    inputs = Input(input_shape)
    # Encoder
    x0_0 = conv_block(inputs, 64)
    p0   = MaxPooling2D()(x0_0)
    x1_0 = conv_block(p0, 128)
    p1   = MaxPooling2D()(x1_0)
    x2_0 = conv_block(p1, 256)
    p2   = MaxPooling2D()(x2_0)
    x3_0 = conv_block(p2, 512)
    p3   = MaxPooling2D()(x3_0)
    x4_0 = conv_block(p3, 1024)

    # Decoder / Nested skip connections
    x3_1 = conv_block(concatenate([
        x3_0,
        Conv2DTranspose(512, (2,2), strides=2, padding="same")(x4_0)
    ]), 512)

    x2_1 = conv_block(concatenate([
        x2_0,
        Conv2DTranspose(256, (2,2), strides=2, padding="same")(x3_0)
    ]), 256)
    x2_2 = conv_block(concatenate([
        x2_0, x2_1,
        Conv2DTranspose(256, (2,2), strides=2, padding="same")(x3_1)
    ]), 256)

    x1_1 = conv_block(concatenate([
        x1_0,
        Conv2DTranspose(128, (2,2), strides=2, padding="same")(x2_0)
    ]), 128)
    x1_2 = conv_block(concatenate([
        x1_0, x1_1,
        Conv2DTranspose(128, (2,2), strides=2, padding="same")(x2_1)
    ]), 128)
    x1_3 = conv_block(concatenate([
        x1_0, x1_1, x1_2,
        Conv2DTranspose(128, (2,2), strides=2, padding="same")(x2_2)
    ]), 128)

    x0_1 = conv_block(concatenate([
        x0_0,
        Conv2DTranspose(64, (2,2), strides=2, padding="same")(x1_0)
    ]), 64)
    x0_2 = conv_block(concatenate([
        x0_0, x0_1,
        Conv2DTranspose(64, (2,2), strides=2, padding="same")(x1_1)
    ]), 64)
    x0_3 = conv_block(concatenate([
        x0_0, x0_1, x0_2,
        Conv2DTranspose(64, (2,2), strides=2, padding="same")(x1_2)
    ]), 64)
    x0_4 = conv_block(concatenate([
        x0_0, x0_1, x0_2, x0_3,
        Conv2DTranspose(64, (2,2), strides=2, padding="same")(x1_3)
    ]), 64)

    outputs = Conv2D(num_classes, (1,1), activation="softmax")(x0_4)
    model = Model(inputs, outputs, name="UNetpp")
    return model

# Example instantiation
unetpp = build_unetplusplus(input_shape=(224,224,3), num_classes=5)
unetpp.summary()


Model: "UNetpp"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_31 (Conv2D)  │ (None, 224, 224,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_31[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_30       │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_32 (Conv2D)  │ (None, 224, 224,  │     36,928 │ activation_30[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_32[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_31       │ (None, 224, 224,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 112, 112,  │          0 │ activation_31[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_33 (Conv2D)  │ (None, 112, 112,  │     73,856 │ max_pooling2d_4[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_33[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_32       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_34 (Conv2D)  │ (None, 112, 112,  │    147,584 │ activation_32[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_34[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_33       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 56, 56,    │          0 │ activation_33[0]… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_35 (Conv2D)  │ (None, 56, 56,    │    295,168 │ max_pooling2d_5[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │      1,024 │ conv2d_35[0][0] 

 Total params: 36,187,397 (138.04 MB)

 Trainable params: 36,172,805 (137.99 MB)

 Non-trainable params: 14,592 (57.00 KB)

# Metrics

In [9]:
import tensorflow as tf

smooth = 1e-15
num_classes = 5  # <-- make this match your final Conv2D(filters=5, ...)

def dice_coefficient(y_true, y_pred):
    # y_true: (B, H, W, 1) integers 0..4
    # y_pred: (B, H, W, 5) softmax probs

    # 1) Squeeze off the last dim → (B, H, W)
    y_true = tf.squeeze(y_true, axis=-1)

    # 2) One‑hot encode → (B, H, W, 5)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)

    # 3) Flatten both to (B*H*W, 5)
    y_true_f = tf.reshape(y_true_o, [-1, num_classes])
    y_pred_f = tf.reshape(y_pred,   [-1, num_classes])

    # 4) Compute intersection & union per class
    intersection = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    union        = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)

    # 5) Dice per class, then average
    dice_per_class = (2. * intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)


#Train


In [12]:
# In a notebook cell, before any cv2 imports:
!pip install --quiet opencv-python-headless
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1) Import TF first and set memory growth (fallback to experimental)
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception as e:
            print(f"[GPU] Could not set memory growth: {e}")
    print(f"[GPU] Detected {len(gpus)} GPU(s).")

# 2) Now import the rest
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger, ReduceLROnPlateau,
    EarlyStopping, TensorBoard
)
from tensorflow.keras.optimizers import Adam

""" Global parameters """
Height = 224
Width  = 224

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path_dataset, val_ratio=0.2, test_ratio=0.1, random_state=42):
    img_dir  = os.path.join(path_dataset, "images")
    mask_dir = os.path.join(path_dataset, "masks")

    image_exts = ('.png', '.jpg', '.jpeg')
    images = sorted([
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_exts)
    ])


    mask_exts = ('.tif', '.tiff')
    masks = sorted([
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if f.lower().endswith(mask_exts)
    ])

    if not images:
        raise RuntimeError(f"No images found in {img_dir}")
    if not masks:
        raise RuntimeError(f"No masks found in {mask_dir}")

    images_lookup = {
        os.path.splitext(os.path.basename(p))[0]: p
        for p in images
    }
#MildDemented_0a7b1321-eba0-40dc-85b8-de34241554a2_jpg.rf.70370fcd9ac0fb82f91d27f558053d84.tif
    pairs = []
    for m in masks:
        base = os.path.splitext(os.path.basename(m))[0]
        if base.endswith('_mask'):
            root = base[:-5]
            img_path = images_lookup.get(root)
            if img_path:
                pairs.append((img_path, m))

    if not pairs:
        raise RuntimeError("No matching image↔mask pairs found!")

    trainval, test = train_test_split(pairs, test_size=test_ratio, random_state=random_state)
    val_frac = val_ratio / (1.0 - test_ratio)
    train, val = train_test_split(trainval, test_size=val_frac, random_state=random_state)

    train_imgs, train_masks = zip(*train)
    val_imgs,   val_masks   = zip(*val)
    test_imgs,  test_masks  = zip(*test)

    return (
        list(train_imgs), list(train_masks),
        list(val_imgs),   list(val_masks),
        list(test_imgs),  list(test_masks)
    )

def read_image_file(path_img):
    path = path_img.decode('utf-8') if isinstance(path_img, bytes) else path_img
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height)) / 255.0
    return img.astype(np.float32)

def read_mask_file(path_mask):
    path = path_mask.decode('utf-8') if isinstance(path_mask, bytes) else path_mask
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[..., 0]
    mask = cv2.resize(mask, (Width, Height), interpolation=cv2.INTER_NEAREST)

    # ✅ Clip to [0, 4], not [0, 3]:
    mask = np.clip(mask, 0, NUM_CLASSES-1)

    # Convert to integer class IDs
    mask = mask.astype(np.int32)

    # Add the channel dim back
    return mask[..., None]

def tf_parse(path_img, path_mask):
    # Tell numpy_function: img→float32, mask→int32
    img, mask = tf.numpy_function(
        lambda pi, pm: (read_image_file(pi), read_mask_file(pm)),
        [path_img, path_mask],
        [tf.float32, tf.int32]          # <-- second output is now int32
    )
    img.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return img, mask

def tf_dataset(inputs, targets, batch_size=2):
    ds = tf.data.Dataset.from_tensor_slices((inputs, targets))
    ds = ds.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def edit_mask_images(mask_dir, func, output_dir=None):
    out_dir = output_dir or mask_dir
    create_folder(out_dir)
    exts = ('.tif')
    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(exts):
            continue
        in_path  = os.path.join(mask_dir, fname)
        mask     = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
        edited   = func(mask)
        cv2.imwrite(os.path.join(out_dir, fname), edited)
    print(f"[edit_mask_images] Applied edits to masks in {out_dir}")

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_folder("/content/drive/MyDrive/Project_41725/files")

    batch_size = 16
    lr         = 1e-4
    epochs_num     = 100
    path_model = os.path.join("/content/drive/MyDrive/Project_41725/files", "model.h5")
    path_csv   = os.path.join("/content/drive/MyDrive/Project_41725/files", "log.csv")

    ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(ds_path)

    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    train_ds = tf_dataset(train_imgs, train_masks, batch_size=batch_size)
    val_ds   = tf_dataset(val_imgs,   val_masks,   batch_size=batch_size)
    test_ds  = tf_dataset(test_imgs,  test_masks,  batch_size=batch_size)


[GPU] Could not set memory growth: Physical devices cannot be modified after being initialized
[GPU] Detected 1 GPU(s).
Train: 1216, Val: 348, Test: 174


In [13]:
import os
import re

mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"

for fname in sorted(os.listdir(mask_dir)):
    if not fname.lower().endswith('.tif'):
        continue

    stem, ext = os.path.splitext(fname)

    if re.search(r'_mask(?:_mask)*$', stem):
        # collapse multiple "_mask" → exactly one
        new_stem = re.sub(r'(?:_mask)+$', '_mask', stem)
    else:
        # no "_mask" suffix at all → add one
        new_stem = stem + '_mask'

    if new_stem != stem:
        new_name = new_stem + ext
        os.rename(
            os.path.join(mask_dir, fname),
            os.path.join(mask_dir, new_name)
        )
        print(f"RENAMED: {fname} → {new_name}")

print("Done.")


Done.


In [14]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
)

# Hyper‑params & paths
NUM_CLASSES = 5
INPUT_SHAPE = (224, 224, 3)
EPOCHS      = 160
path_model  = "/content/drive/MyDrive/Project_41725/files/model.h5"
path_csv    = "/content/drive/MyDrive/Project_41725/files/log.csv"

# 0) kill any existing graph/state
tf.keras.backend.clear_session()

# 1) rebuild
model = build_unetplusplus(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

# 2) override its internal name (strip out the '+')
model._name = "unetpp"

# 3) Define losses & metrics
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=NUM_CLASSES)
    y_true_f = tf.reshape(y_true_o, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred,   [-1, NUM_CLASSES])
    inter = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    denom = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * inter + smooth) / (denom + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

class MeanIoUArgMax(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes=NUM_CLASSES, name="mean_iou", dtype=None):
        super().__init__(num_classes=num_classes, name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

# 4) Compile
model.compile(
    optimizer=Adam(1e-4),
    loss=combined_loss,
    metrics=[dice_coef, MeanIoUArgMax()]
)


# 5) Callbacks
callbacks = [
    ModelCheckpoint(path_model, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(path_csv),
    EarlyStopping(monitor='val_loss', patience=20),
]

# 6) (Optional) Evaluate before training
print("Before training:", dict(zip(model.metrics_names, model.evaluate(test_ds))))

# 7) Fit
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


11/11 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - dice_coef: 0.1129 - loss: 2.4845 - mean_iou: 0.1278
Before training: {'loss': 2.4850287437438965, 'compile_metrics': 0.11227607727050781}
Epoch 1/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - dice_coef: 0.2907 - loss: 1.3691 - mean_iou: 0.2935
Epoch 1: val_loss improved from inf to 2.05561, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 308s 3s/step - dice_coef: 0.2912 - loss: 1.3653 - mean_iou: 0.2941 - val_dice_coef: 0.1485 - val_loss: 2.0556 - val_mean_iou: 0.1455 - learning_rate: 1.0000e-04
Epoch 2/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 963ms/step - dice_coef: 0.3674 - loss: 0.8463 - mean_iou: 0.3664
Epoch 2: val_loss did not improve from 2.05561
76/76 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - dice_coef: 0.3674 - loss: 0.8460 - mean_iou: 0.3664 - val_dice_coef: 0.1601 - val_loss: 2.2661 - val_mean_iou: 0.1455 - learning_rate: 1.0000e-04
Epoch 3/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 886ms/step - dice_coef: 0.3810 - loss: 0.7798 - mean_iou: 0.3832
Epoch 3: val_loss did not improve from 2.05561
76/76 ━━━━━━━━━━━━━━━━━━━━ 74s 971ms/step - dice_coef: 0.3811 - loss: 0.7796 - mean_iou: 0.3833 - val_dice_coef: 0.1682 - val_loss: 2.4218 - val_mean_iou: 0.1455 - learning_rate: 1.0000e-04
Epoch 4/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 888ms/step - dice_coef: 0.3954 - loss: 0.7353 - mean_iou: 0.4064
Epoch 4: val_loss di

76/76 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - dice_coef: 0.5038 - loss: 0.5657 - mean_iou: 0.5711 - val_dice_coef: 0.3830 - val_loss: 0.9727 - val_mean_iou: 0.3578 - learning_rate: 1.0000e-05
Epoch 8/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 965ms/step - dice_coef: 0.5237 - loss: 0.5384 - mean_iou: 0.6051
Epoch 8: val_loss improved from 0.97268 to 0.59311, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - dice_coef: 0.5238 - loss: 0.5383 - mean_iou: 0.6052 - val_dice_coef: 0.4999 - val_loss: 0.5931 - val_mean_iou: 0.5771 - learning_rate: 1.0000e-05
Epoch 9/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 960ms/step - dice_coef: 0.5353 - loss: 0.5239 - mean_iou: 0.6224
Epoch 9: val_loss improved from 0.59311 to 0.54576, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - dice_coef: 0.5354 - loss: 0.5237 - mean_iou: 0.6226 - val_dice_coef: 0.5306 - val_loss: 0.5458 - val_mean_iou: 0.6224 - learning_rate: 1.0000e-05
Epoch 10/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 959ms/step - dice_coef: 0.5482 - loss: 0.5085 - mean_iou: 0.6551
Epoch 10: val_loss improved from 0.54576 to 0.52159, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


...

76/76 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - dice_coef: 0.7976 - loss: 0.2236 - mean_iou: 0.9134 - val_dice_coef: 0.7230 - val_loss: 0.3283 - val_mean_iou: 0.7420 - learning_rate: 1.0000e-05
Epoch 40/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 955ms/step - dice_coef: 0.8015 - loss: 0.2193 - mean_iou: 0.9128
Epoch 40: val_loss improved from 0.32826 to 0.32680, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - dice_coef: 0.8015 - loss: 0.2193 - mean_iou: 0.9125 - val_dice_coef: 0.7252 - val_loss: 0.3268 - val_mean_iou: 0.7395 - learning_rate: 1.0000e-05
Epoch 41/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 967ms/step - dice_coef: 0.8048 - loss: 0.2157 - mean_iou: 0.9126
Epoch 41: val_loss did not improve from 0.32680
76/76 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - dice_coef: 0.8049 - loss: 0.2157 - mean_iou: 0.9123 - val_dice_coef: 0.7240 - val_loss: 0.3276 - val_mean_iou: 0.7378 - learning_rate: 1.0000e-05
Epoch 42/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 886ms/step - dice_coef: 0.8101 - loss: 0.2098 - mean_iou: 0.9169
Epoch 42: val_loss did not improve from 0.32680
76/76 ━━━━━━━━━━━━━━━━━━━━ 74s 973ms/step - dice_coef: 0.8101 - loss: 0.2099 - mean_iou: 0.9166 - val_dice_coef: 0.7169 - val_loss: 0.3338 - val_mean_iou: 0.7275 - learning_rate: 1.0000e-05
Epoch 43/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 888ms/step - dice_coef: 0.8151 - loss: 0.2043 - mean_iou: 0.9205
Epoch 43: val_lo

76/76 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - dice_coef: 0.8151 - loss: 0.2043 - mean_iou: 0.9202 - val_dice_coef: 0.7291 - val_loss: 0.3254 - val_mean_iou: 0.7329 - learning_rate: 1.0000e-05
Epoch 44/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 964ms/step - dice_coef: 0.8173 - loss: 0.2020 - mean_iou: 0.9175
Epoch 44: val_loss did not improve from 0.32543
76/76 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - dice_coef: 0.8172 - loss: 0.2020 - mean_iou: 0.9172 - val_dice_coef: 0.7245 - val_loss: 0.3327 - val_mean_iou: 0.7241 - learning_rate: 1.0000e-05
Epoch 45/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 887ms/step - dice_coef: 0.8039 - loss: 0.2175 - mean_iou: 0.8887
Epoch 45: val_loss improved from 0.32543 to 0.30557, saving model to /content/drive/MyDrive/Project_41725/files/model.h5


76/76 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - dice_coef: 0.8040 - loss: 0.2173 - mean_iou: 0.8886 - val_dice_coef: 0.7442 - val_loss: 0.3056 - val_mean_iou: 0.7499 - learning_rate: 1.0000e-05
Epoch 46/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 962ms/step - dice_coef: 0.8213 - loss: 0.1979 - mean_iou: 0.9124
Epoch 46: val_loss did not improve from 0.30557
76/76 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - dice_coef: 0.8213 - loss: 0.1979 - mean_iou: 0.9121 - val_dice_coef: 0.7232 - val_loss: 0.3263 - val_mean_iou: 0.7271 - learning_rate: 1.0000e-05
Epoch 47/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 888ms/step - dice_coef: 0.8273 - loss: 0.1913 - mean_iou: 0.9184
Epoch 47: val_loss did not improve from 0.30557
76/76 ━━━━━━━━━━━━━━━━━━━━ 74s 974ms/step - dice_coef: 0.8273 - loss: 0.1913 - mean_iou: 0.9181 - val_dice_coef: 0.7288 - val_loss: 0.3217 - val_mean_iou: 0.7280 - learning_rate: 1.0000e-05
Epoch 48/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 889ms/step - dice_coef: 0.8327 - loss: 0.1851 - mean_iou: 0.9259
Epoch 48: val_lo

76/76 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - dice_coef: 0.8384 - loss: 0.1785 - mean_iou: 0.9227 - val_dice_coef: 0.7581 - val_loss: 0.2903 - val_mean_iou: 0.7614 - learning_rate: 1.0000e-06
Epoch 52/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 967ms/step - dice_coef: 0.8454 - loss: 0.1707 - mean_iou: 0.9367
Epoch 52: val_loss did not improve from 0.29029
76/76 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - dice_coef: 0.8454 - loss: 0.1707 - mean_iou: 0.9364 - val_dice_coef: 0.7572 - val_loss: 0.2911 - val_mean_iou: 0.7605 - learning_rate: 1.0000e-06
Epoch 53/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 887ms/step - dice_coef: 0.8461 - loss: 0.1699 - mean_iou: 0.9376
Epoch 53: val_loss did not improve from 0.29029
76/76 ━━━━━━━━━━━━━━━━━━━━ 74s 973ms/step - dice_coef: 0.8461 - loss: 0.1699 - mean_iou: 0.9373 - val_dice_coef: 0.7565 - val_loss: 0.2916 - val_mean_iou: 0.7594 - learning_rate: 1.0000e-06
Epoch 54/160
76/76 ━━━━━━━━━━━━━━━━━━━━ 0s 889ms/step - dice_coef: 0.8467 - loss: 0.1692 - mean_iou: 0.9383
Epoch 54: val_lo

#Test


In [ ]:
from google.colab import drive
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab import files

# 1) Mount drive
drive.mount('/content/drive', force_remount=True)

# 2) Make sure your dice functions are in scope if you ever re‑compile
# def dice_coefficient(...): ...
# def dice_coefficient_loss(...): ...

model_path = "/content/drive/MyDrive/Project_41725/files/model.h5"

# 3) Load model for inference only—skip compile to avoid the TypeError
model = load_model(
    model_path,
    custom_objects={
        'dice_coefficient': dice_coefficient,
        'dice_coefficient_loss': dice_coefficient_loss
    },
    compile=False
)

def predict_and_show(img_path, model, input_size=(224,224)):
    gray    = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    resized = cv2.resize(gray, input_size)
    x       = np.stack([resized]*3, -1).astype(np.float32) / 255.0

    # Run model
    pred = model.predict(x[None])[0]         # ➔ (H, W, num_classes)
    seg  = pred.argmax(-1).astype(np.uint8)  # ➔ (H, W)

    # Compute and print class percentages
    unique, counts = np.unique(seg, return_counts=True)
    total = seg.size
    percentages = {int(cls): float(cnt/total*100) for cls, cnt in zip(unique, counts)}
    print(f"\nClass percentages for {img_path}:")
    for cls, pct in percentages.items():
        print(f"  Class {cls}: {pct:.2f}%")

    # Display images
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.title("Input")
    plt.imshow(resized, cmap='gray')
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.title("Segmentation Overlay")
    plt.imshow(resized, cmap='gray', alpha=0.5)
    plt.imshow(seg, cmap='jet',   alpha=0.5)
    plt.axis('off')
    plt.show()

# 4) Upload & run on uploaded files
uploaded = files.upload()
for fn in uploaded:
    predict_and_show(fn, model)
    print("Num output channels:", model.output_shape[-1])


In [ ]:
import matplotlib.pyplot as plt
# pick a few file paths from train_masks
for mpath in train_masks[:5]:
    mask = cv2.imread(mpath, cv2.IMREAD_GRAYSCALE)
    plt.figure(figsize=(4,4))
    plt.title(mpath.split("/")[-1])
    plt.imshow(mask, cmap="tab10", vmin=0, vmax=3)
    plt.colorbar(ticks=[0,1,2,3,4])
    plt.show()


In [ ]:
import numpy as np

counts = np.zeros(4, dtype=int)
for mpath in train_masks:
    mask = cv2.imread(mpath, cv2.IMREAD_GRAYSCALE).flatten()
    for c in range(4):
        counts[c] += np.sum(mask == c)

freqs = counts / counts.sum()
print("Class frequencies:", {i: freqs[i] for i in range(4)})


In [ ]:
import tifffile
import numpy as np

for mpath in train_masks[:2000]:
    mask = tifffile.imread(mpath)            # read exact IDs
    print(mpath, np.unique(mask))


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

""" Global parameters """
Height = 224
Width = 224

""" Creating a directory in drive """
def create_folder(path):
    """
    Create a directory if it does not exist.

    Args:
        path: Path to the directory.
    """
    if not os.path.exists(path):
        os.makedirs(path)

def Result_saving(inp_image, m_true, p_mask, path_saved_image):
    """
    Save the image, ground truth mask, and predicted mask side by side.

    Args:
        inp_image: Input image.
        m_true: Ground truth mask.
        p_mask: Predicted mask.
        path_saved_image: Path to save the image.
    """
    m_true  =  np.expand_dims ( m_true, axis=-1)
    m_true  =  np.concatenate ( [ m_true, m_true, m_true], axis=-1 )

    p_mask = np.expand_dims ( p_mask, axis=-1)
    p_mask = np.concatenate ( [ p_mask, p_mask, p_mask], axis=-1)
    p_mask = p_mask * 255

    line = np.ones((Height, 10, 3)) * 255

    concat_images = np.concatenate([inp_image, line, m_true, line, p_mask], axis=1)
    cv2.imwrite(path_saved_image, concat_images )

if __name__ == "__main__":
    """ Seeding the tensor """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_folder("/content/drive/MyDrive/DSP Project/results")

    from tensorflow.keras.models import load_model

# Define custom metrics and loss
    def dice_coefficient(y_true, y_pred, smooth=1e-6):
      y_true_f = tf.keras.backend.flatten(y_true)
      y_pred_f = tf.keras.backend.flatten(y_pred)
      intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
      return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

    def dice_coefficient_loss(y_true, y_pred):
      return 1.0 - dice_coefficient(y_true, y_pred)

    # Define this dictionary BEFORE calling load_model
    custom_objects = {
         "dice_coefficient": dice_coefficient,
         "dice_coefficient_loss": dice_coefficient_loss
      }

    # Now safely load the model
    model_loaded = load_model("/content/drive/MyDrive/Project_41725/files/model.h5", custom_objects=custom_objects)

    """ loading the dataset from drive"""
    path_dataset ="/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(path_dataset)


    """ Model Prediction and Evaluation along with score saving to drive """
    EVL_SCORE_list = []
    for im, msk in tqdm(zip(test_imgs, test_masks), total=len(test_masks)):
        """ Extracting the name of image """
        n_img = im.split("/")[-1]

        """ Reading the image """
        inp_image = cv2.imread(im, cv2.IMREAD_COLOR) ## [Height, Width, 3]
        inp_image = cv2.resize(inp_image, (Width, Height))       ## [Height, Width, 3]
        im = inp_image/255.0                         ## [Height, Width, 3]
        im = np.expand_dims(im, axis=0)           ## [1, Height, Width, 3]

        """ Reading the mask """
        m_true = cv2.imread(msk, cv2.IMREAD_GRAYSCALE)
        m_true = cv2.resize(m_true, (Width, Height))

        """ Predicting mask """
        p_mask = model_loaded.predict(im, verbose=0)[0]              # Output shape: (224, 224, 4)
        p_mask = np.argmax(p_mask, axis=-1).astype(np.uint8)         # Convert softmax to class ID


        """ Save the predicting mask result """
        path_saved_image = os.path.join("/content/drive/MyDrive/DSP Project/results", n_img)
        Result_saving(inp_image, m_true, p_mask, path_saved_image)

        """ Flatten the array """
        m_true = m_true.astype(np.uint8).flatten()
        p_mask = p_mask.flatten()

        """ finding the evalution value """
        f1_scoree        = f1_score(m_true, p_mask.flatten(), average="macro")
        jaccard_scoree   = jaccard_score(m_true, p_mask.flatten(), average="macro")
        recall_scoree    = recall_score(m_true, p_mask.flatten(), average="macro", zero_division=0)
        precision_scoree = precision_score(m_true, p_mask.flatten(), average="macro", zero_division=0)


    """ EEvluation Matrix """
    evl_score_val = [s[1:]for s in EVL_SCORE_list]
    evl_score_val = np.mean(evl_score_val, axis=0)
    print(f"F1: {evl_score_val[0]:0.5f}")
    print(f"Jaccard: {evl_score_val[1]:0.5f}")
    print(f"Recall: {evl_score_val[2]:0.5f}")
    print(f"Precision: {evl_score_val[3]:0.5f}")

    df = pd.DataFrame(EVL_SCORE_list, columns=["Image", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("/content/drive/MyDrive/DSP Project/files/score.csv")


#Plot and Checks

In [ ]:
import os
import shutil

def move_images(src_folder, dst_folder):
    """
    Move all image files from src_folder to dst_folder.

    Args:
        src_folder (str): Source directory containing image files.
        dst_folder (str): Destination directory where images will be moved.
    """
    os.makedirs(dst_folder, exist_ok=True)
    image_exts = ('.png', '.jpg', '.jpeg')

    moved = 0
    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith(image_exts):
            src_path = os.path.join(src_folder, file_name)
            dst_path = os.path.join(dst_folder, file_name)
            shutil.move(src_path, dst_path)
            moved += 1

    print(f"[INFO] Moved {moved} image(s) from {src_folder} → {dst_folder}")

move_images("/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks", "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/ignore")


In [ ]:
import pandas as pd
metrics = pd.read_csv("/content/drive/MyDrive/DSP Project/files/log.csv")
metrics.head(10)

In [ ]:
metrics[['dice_coef','val_dice_coef']].plot()

In [ ]:
metrics[['loss','val_loss']].plot()

In [ ]:
scores = pd.read_csv("/content/drive/MyDrive/DSP Project/files/score.csv")
scores.head(10)

#Watershed implementation for automataing segmentation

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.segmentation import watershed as skwater
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

def ShowImage(list_img, list_t, list_cm):
    """
    Display multiple images with corresponding titles and colormaps.

    Args:
        list_img (list): List of images to be displayed.
        list_t (list): List of titles for each image.
        list_cm (list): List of colormaps for each image.
    """
    no_of_img = len(list_img)
    figure, axis_label = plt.subplots(1, no_of_img, figsize = (10*no_of_img, 10))
    for x  in range(no_of_img):
        axis_label[x].imshow(list_img[x], cmap=list_cm[x])
        axis_label[x].axis('off')
        axis_label[x].set_title(list_t[x])
    plt.show()

def watershed(path_img):
    """
    Apply watershed segmentation to an image.

    Args:
        path_img (str): Path to the input image.
    """
    # Read the image
    color_image = cv2.imread(path_img)
    grey_img = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to obtain a binary image
    computed_threshold, threshold = cv2.threshold(grey_img, 0, 255, cv2.THRESH_OTSU)
    computed_threshold, labeled_image = cv2.connectedComponents(threshold)

    # Find the largest component (presumed to be the brain)
    marker_area = [np.sum(labeled_image == m) for m in range(1, np.max(labeled_image))]
    largest_component = np.argmax(marker_area) + 1
    foreground = labeled_image == largest_component
    brain_out = color_image.copy()
    brain_out[foreground == False] = (0, 0, 0)

    # Apply morphological operations to refine the segmentation
    grey_img = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    computed_threshold, threshold = cv2.threshold(grey_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel, iterations=2)
    background = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    computed_threshold, transform_threshold = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    transform_threshold = np.uint8(transform_threshold)
    unknown = cv2.subtract(background, transform_threshold)
    computed_threshold, labeled_image = cv2.connectedComponents(transform_threshold)
    labeled_image = labeled_image + 1
    labeled_image[unknown == 255] = 0

    # Perform watershed segmentation
    labeled_image = cv2.watershed(color_image, labeled_image)
    color_image[labeled_image == -1] = [255, 0, 0]
    im1 = cv2.cvtColor(color_image, cv2.COLOR_HSV2RGB)

    # Apply morphological closing to refine the brain mask
    foreground = np.uint8(foreground)
    kernel = np.ones((8, 8), np.uint8)
    closing = cv2.morphologyEx(foreground, cv2.MORPH_CLOSE, kernel)

    brain_out = color_image.copy()
    brain_out[closing == False] = (0, 0, 0)

    # Prepare images, titles, and colormaps for visualization
    images = [grey_img, threshold, im1, closing]
    titles = ['Brain MRI', 'Thresholding image', 'Watershed segmented image', 'Closing']
    cmaps = ['gray', 'gray', 'gray', 'gray']

    # Display the images
    ShowImage(images, titles, cmaps)


#AI VS DSP

In [ ]:
import matplotlib.pyplot as plt


dir = "/content/drive/MyDrive/DSP Project/results/"
images = os.listdir("/content/drive/MyDrive/DSP Project/results/")[:15]
a  = "33.png"
img1 = plt.imread(dir+ a)
plt.imshow(img1, cmap='gray')

img_path = "/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a
print(watershed("/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a))

In [ ]:
import matplotlib.pyplot as plt


dir = "/content/drive/MyDrive/DSP Project/results/"
images = os.listdir("/content/drive/MyDrive/DSP Project/results/")[:15]
a  = images[2]
img1 = plt.imread(dir+ a)
plt.imshow(img1, cmap='gray')

img_path = "/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a
print(watershed("/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a))

In [ ]:
import matplotlib.pyplot as plt


dir = "/content/drive/MyDrive/DSP Project/results/"
images = os.listdir("/content/drive/MyDrive/DSP Project/results/")[:15]
a  = images[10]
img1 = plt.imread(dir+ a)
plt.imshow(img1, cmap='gray')

img_path = "/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a
print(watershed("/content/drive/MyDrive/DSP Project/brain_tumor_dataset/data/images/" +a))